# Collaborative Filtering Recommendation

Use pyspark and sql in databricks to build a recommendation system and predict the rating scores for users in the test and recommend songs with top rate

## Bring in data and EDA

In [3]:
# coding=utf-8
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, FloatType
from pyspark.sql.functions import lit
from pyspark.sql.functions import trim
from pyspark.sql.functions import round
from pyspark.sql.functions import when
import pyspark.sql.functions as func
spark = SparkSession \
    .builder \
    .appName("music player") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
spark.catalog.clearCache()

schema_all_play = StructType([ StructField("uid", StringType(), True), StructField("device", StringType(), True),StructField("song_id", StringType(), True), StructField("song_type", StringType(), True), StructField("song_name", StringType(), True), StructField("singer", StringType(), True), StructField("play_time", StringType(), True),StructField("song_length", StringType(), True), StructField("paid_flag", StringType(), True), StructField("date", StringType(), True)])

schema_play = StructType([ StructField("uid", StringType(), True), StructField("device", StringType(), True),StructField("song_id", StringType(), True), StructField("song_type", StringType(), True), StructField("song_name", StringType(), True), StructField("singer", StringType(), True), StructField("play_time", StringType(), True),StructField("song_length", StringType(), True), StructField("paid_flag", StringType(), True)])

all_play = spark.createDataFrame([], schema = schema_all_play)

  
for path, name, _ in dbutils.fs.ls('/FileStore/tables/play/'):
# for path, name, _ in dbutils.fs.ls('/FileStore/tables/gz/'): 
  df = spark.read.csv(path, sep = "\t", schema = schema_play)
  date = name.split("_")[0].strip()
  df = df.withColumn("date",lit(date))

  all_play = all_play.unionAll(df)
# all_play = all_play.filter(all_play['play_time'] > 0.8 * all_play['song_length'] )
# all_play.cache()
# print(all_play.count())
all_play.createOrReplaceTempView("all_play")

In [4]:
all_play.select('uid').distinct().count()

Out[ 5 ]: 594563

In [5]:
all_play.cache()

Out[ 2 ]: DataFrame[uid: string, device: string, song_id: string, song_type: string, song_name: string, singer: string, play_time: string, song_length: string, paid_flag: string, date: string]

In [6]:
all_play.select('song_id').distinct().count()

Out[ 3 ]: 1564951

In [7]:
all_play.select('song_name').distinct().count()

Out[ 4 ]: 1615713

In [8]:
n = spark.sql("""
SELECT cast(uid as Integer), cast(song_id as Integer), (case when rk >= 0.8 then 3 when rk <= 0.3 then 1 else 2 end)  as rate
FROM 
(
SELECT  uid, song_id, cnt_song_uid,  percent_rank(cnt_song_uid) over(partition by uid order by cnt_song_uid) as rk , count(uid) over(partition by song_id) as cnt_uid
FROM (
SELECT uid, song_id, count(*) as cnt_song_uid  FROM 
(
SELECT uid, song_id,  play_time,  date, first_value(song_length) over(order by cnt_song desc) as song_length, first_value(cnt_date) over(partition by uid order by cnt_date desc) as max
FROM 
(
SELECT uid, song_id,  play_time, song_length, date, count(*) over(partition by song_id, song_length) as cnt_song, dense_rank() over(order by uid, date)  as cnt_date
FROM ALL_PLAY
WHERE singer > 'z' and substring(singer, -1, 1) > 'z' and substring(singer, -1, 1) not in ('手', '曲', '谦', '妍', '刚', '阳', '乐', '纬', '伦', '星', '雷',  '声',  '然', '歌', '团', '事', '物') and play_time > 0.8*song_length and uid not in (1685126, 751824, 37025504, 1791497, 1062806, 497685, 751824, 37025504, 1791497, 1062806, 497685, 736305, 0, 1749320, 1679121, 46532274, 28638487, 533817, 155948236, 637650, 32166203, 64268006, 26036032, 32104145, 398309, 154539052, 16517426, 27954505, 168095004, 1883192, 1710083, 1963913, 924065, 167615175, 167848561, 168565798, 168510160) and song_id > 9999 and song_id not in (5237384, 13273544, 15807836)
) t1
) t2 WHERE max > 25 and song_length > 100 and song_length < 480 and ( (play_time > 999 and play_time/1000>0.8*song_length)  or play_time < 1.1*song_length)
GROUP BY uid, song_id
)h
) t4
WHERE cnt_uid > 1
""")

In [9]:
all_play.cache()
all_play.count()

Out[ 3 ]: 145791789

In [10]:
n.cache()
n.count()

Out[ 4 ]: 8904026

## Collaborative Filtering

In [11]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
training, testing = n.randomSplit([0.8, 0.2])
als = ALS(maxIter = 5, regParam = 0.01, userCol = 'uid', itemCol = 'song_id', ratingCol = 'rate')
model = als.fit(training)
predictions = model.transform(testing)
evaluator = RegressionEvaluator(metricName =  'rmse', labelCol = 'rate', predictionCol = 'prediction')


In [12]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName =  'rmse', labelCol = 'rate', predictionCol = 'prediction')

predictions = predictions.where("prediction != 'NaN'")
rmse = evaluator.evaluate(predictions)

In [13]:
evaluator.evaluate(predictions)

Out[ 9 ]: 0.8703438830005178

In [ ]:
# predict songs' rate for users in the prediction dataset 
# songs with top rating scores are recommended  
# other data explorations are as follows

## Other EDA

In [14]:
# other data exploration
dbutils.fs.rm("/FileStore/tables/play/20170301_play.log")
dbutils.fs.rm("/FileStore/tables/play/20170339_1_play.log")
for i in range(1,10):
  print(dbutils.fs.rm("/FileStore/tables/play/2017030" + str(i) + "_1_play.log") )

False
True
True
True
True
True
True
False
True

In [18]:
# coding=utf-8
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, FloatType
from pyspark.sql.functions import lit
from pyspark.sql.functions import trim
from pyspark.sql.functions import round
from pyspark.sql.functions import when
import pyspark.sql.functions as func
spark = SparkSession \
    .builder \
    .appName("music player") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
spark.catalog.clearCache()

schema_all_play = StructType([ StructField("uid", StringType(), True), StructField("device", StringType(), True),StructField("song_id", StringType(), True), StructField("song_type", StringType(), True), StructField("song_name", StringType(), True), StructField("singer", StringType(), True), StructField("play_time", StringType(), True),StructField("song_length", StringType(), True), StructField("paid_flag", StringType(), True), StructField("date", StringType(), True)])

schema_play = StructType([ StructField("uid", StringType(), True), StructField("device", StringType(), True),StructField("song_id", StringType(), True), StructField("song_type", StringType(), True), StructField("song_name", StringType(), True), StructField("singer", StringType(), True), StructField("play_time", StringType(), True),StructField("song_length", StringType(), True), StructField("paid_flag", StringType(), True)])

all_play = spark.createDataFrame([], schema = schema_all_play)

  
for path, name, _ in dbutils.fs.ls('/FileStore/tables/play/'):
# for path, name, _ in dbutils.fs.ls('/FileStore/tables/gz/'): 
  df = spark.read.csv(path, sep = "\t", schema = schema_play)
  date = name.split("_")[0].strip()
  df = df.withColumn("date",lit(date))

  all_play = all_play.unionAll(df)

all_play.createOrReplaceTempView("all_play")

In [19]:
%sql
SELECT COUNT(*)
FROM all_play
WHERE play_time > 0.8*song_length

count(1)
88402275


In [20]:
all_play.count()
# with 0.8 rule dropping

Out[ 5 ]: 88402275

In [21]:
all_play.groupby(func.substring(all_play['uid'],1, 1)).count().show()
# another test 4.09min 1.06 min, why so fast?

+--------------------+--------+
substring(uid, 1, 1)| count|
+--------------------+--------+
 7| 4959620|
 3| 3984873|
 8| 80647|
 0| 777430|
 5| 280909|
 6| 342013|
 9| 95713|
 1|75345445|
 4| 2138667|
 2| 396958|
+--------------------+--------+

In [22]:
# coding=utf-8
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, FloatType
from pyspark.sql.functions import lit
from pyspark.sql.functions import trim
from pyspark.sql.functions import round
from pyspark.sql.functions import when
import pyspark.sql.functions as func
spark = SparkSession \
    .builder \
    .appName("music player") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
spark.catalog.clearCache()

schema_all_play = StructType([ StructField("uid", StringType(), True), StructField("device", StringType(), True),StructField("song_id", StringType(), True), StructField("song_type", StringType(), True), StructField("song_name", StringType(), True), StructField("singer", StringType(), True), StructField("play_time", StringType(), True),StructField("song_length", StringType(), True), StructField("paid_flag", StringType(), True), StructField("date", StringType(), True)])

schema_play = StructType([ StructField("uid", StringType(), True), StructField("device", StringType(), True),StructField("song_id", StringType(), True), StructField("song_type", StringType(), True), StructField("song_name", StringType(), True), StructField("singer", StringType(), True), StructField("play_time", StringType(), True),StructField("song_length", StringType(), True), StructField("paid_flag", StringType(), True)])

all_play = spark.createDataFrame([], schema = schema_all_play)

  
for path, name, _ in dbutils.fs.ls('/FileStore/tables/play/'):
# for path, name, _ in dbutils.fs.ls('/FileStore/tables/gz/'): 
  df = spark.read.csv(path, sep = "\t", schema = schema_play)
  date = name.split("_")[0].strip()
  df = df.withColumn("date",lit(date))

  all_play = all_play.unionAll(df)
all_play = all_play.filter(all_play['play_time'] > 0.8 * all_play['song_length'] )
# all_play.cache()
# print(all_play.count())
all_play.createOrReplaceTempView("all_play")

In [23]:
all_play = all_play.filter( (all_play.uid != 1685126) \
                    &(all_play.uid != 751824) \
                    &(all_play.uid != 37025504) \
                    &(all_play.uid != 1791497) \
                    &(all_play.uid != 1062806) \
                    &(all_play.uid != 497685) \
                    &(all_play.uid != 736305) \
                    &(all_play.uid > 99999) \
                    &(all_play.uid != 1749320) \
                    &(all_play.uid != 1679121) \
                    &(all_play.uid != 46532274) \
                    &(all_play.uid != 28638487) \
                    &(all_play.uid != 533817) \
                    &(all_play.uid != 155948236) \
                    &(all_play.uid != 637650) \
                    &(all_play.uid != 32166203) \
                    &(all_play.uid != 64268006) \
                    &(all_play.uid != 26036032) \
                    &(all_play.uid != 32104145) \
                    &(all_play.uid != 398309) \
                    &(all_play.uid != 154539052)\
                    & (all_play.uid != 16517426)\
                    & (all_play.uid != 27954505)\
                    & (all_play.uid != 168095004)\
                    & (all_play.uid != 1710083)\
                    & (all_play.uid != 1963913)\
                    & (all_play.uid != 924065)\
                    & (all_play.uid != 167615175)\
                    & (all_play.uid != 167848561)\
                    & (all_play.uid != 168565798)\
                    & (all_play.uid != 168510160)  )

In [24]:
all_play.cache()

Out[ 2 ]: DataFrame[uid: string, device: string, song_id: string, song_type: string, song_name: string, singer: string, play_time: string, song_length: string, paid_flag: string, date: string]

In [25]:
all_play.groupby('uid').agg(func.countDistinct('song_id').alias('count')).orderBy('count', ascending = False).show(1000)

+----------+------+
 uid| count|
+----------+------+
 1685126|188188|
 37025504|137595|
 751824|116853|
 736305| 79141|
 1685126 | 78867|
 1062806| 76762|
 497685| 75671|
 1791497| 65220|
 37025504 | 56540|
 0| 49527|
 1749320| 30569|
 1679121| 22880|
 46532274| 22768|
 637650| 21410|
 28638487| 15892|
 533817| 13174|
 26036032 | 12356|
 1749320 | 11703|
 398309 | 10179|
 155948236| 9854|
 0 | 9637|
 751824 | 9530|
 736305 | 9181|
 32104145| 8453|
 1883192 | 7398|
 32166203| 7346|
 1677249 | 6515|
 1710083| 6463|
 27954505| 6359|
 28638487 | 6200|
 1791497 | 6198|
 64268006| 6159|
 533817 | 4971|
 16517426| 3638|
 168915463| 3610|
 45888463| 3189|
 69546515| 2983|
 924065| 2834|
 169031835| 2816|
 167856245| 2608|
 46532274 | 2553|
 168179842| 2552|
 168884001| 2398|
 168377676| 2372|
 168412815| 2362|
 1683241| 2143|
 32166203 | 2094|
 168278040| 2064|
 45365429| 1987|
 168343454| 1973|
 168224877| 1929|
 1679121 | 1881|
 168308159| 1863|
 168611409| 1832|
 168849784| 1831|
 168275605| 1824|
 168788690| 1815|
 168287265| 1806|
 168409118| 1789|
 153049341| 1765|
 167569705| 1750|
 168163522| 1730|
 637650 | 1651|
 167754128| 1644|
 167791348| 1644|
 168911999| 1640|
 42236906| 1638|
 167718052| 1635|
 167849958| 1629|
 1963913| 1629|
 168251426| 1617|
 167898102| 1616|
 168418837| 1582|
 167722439| 1582|
 43658018| 1575|
 167884496| 1525|
 168224985| 1498|
 168451768| 1487|
 168301741| 1485|
 168702360| 1482|
 168262301| 1455|
 168309076| 1454|
 168461660| 1437|
 168714904| 1420|
 167808656| 1417|
 167839399| 1399|
 168139162| 1397|
 167805456| 1396|
 168280987| 1396|
 168988429| 1393|
 168595606| 1353|
 168836057| 1350|
 168536012| 1336|
 167765223| 1326|
 168051193| 1319|
 168483009| 1315|
 168156556| 1310|
 168529587| 1307|
 168178214| 1305|
 33513144| 1301|
 168250614| 1297|
 168060852| 1289|
 168938401| 1287|
 168645368| 1287|
 62692715| 1286|
 168066160| 1285|
 167587977| 1281|
 168368751| 1280|
 168621058| 1279|
 168639534| 1270|
 45312687| 1269|
 167801421| 1268|
 168356685| 1255|
 168442115| 1254|
 167880374| 1252|
 168583145| 1251|
 168157845| 1251|
 65116455| 1240|
 168052947| 1236|
 168155945| 1233|
 168414339| 1231|
 168355190| 1229|
 167596452| 1226|
 58066741| 1219|
 167772970| 1217|
 168636306| 1212|
 167691217| 1210|
 168479098| 1209|
 167737081| 1204|
 168486395| 1186|
 168497211| 1182|
 168015735| 1181|
 168542433| 1180|
 168001008| 1180|
 169038211| 1167|
 168955269| 1167|
 168751924| 1166|
 167826027| 1151|
 155180961| 1149|
 167728979| 1148|
 168045955| 1148|
 168466886| 1143|
 168289797| 1138|
 168755173| 1129|
 168355686| 1129|
 167820900| 1127|
 167739237| 1124|
 168283992| 1121|
 168909986| 1119|
 167792930| 1108|
 167967073| 1108|
 28146229| 1104|
 168491032| 1103|
 168027704| 1095|
 168052175| 1091|
 168192825| 1084|
 168164135| 1080|
 167775152| 1070|
 167663420| 1066|
 168606686| 1065|
 167983827| 1060|
 167727609| 1060|
 168930795| 1057|
 168954949| 1056|
 168683103| 1055|
 167910817| 1054|
 168523030| 1053|
 167834042| 1048|
 168865280| 1048|
 167758319| 1038|
 168822909| 1033|
 25242167| 1030|
 60786265| 1027|
 167575392| 1026|
 168707975| 1023|
 168387210| 1022|
 42233120| 1022|
 168674358| 1015|
 168254653| 1014|
 169041722| 1014|
 168658656| 1012|
 168637523| 1005|
 168462238| 1004|
 168345192| 999|
 167846506| 995|
 168157255| 994|
 167906119| 990|
 167763847| 989|
 168188112| 985|
 168661744| 985|
 168355960| 982|
 167706804| 981|
 169028588| 981|
 167835950| 980|
 168357183| 979|
 169024129| 976|
 168051149| 975|
 168539380| 973|
 168266329| 968|
 167895141| 968|
 168150083| 962|
 168798094| 962|
 168733141| 960|
 169006057| 960|
 168345004| 954|
 167640838| 954|
 167952240| 953|
 168652062| 952|
 168893573| 952|
 168792308| 950|
 167689442| 949|
 168827140| 948|
 164115461| 947|
 168726962| 940|
 57820269| 940|
 167776982| 939|
 168006485| 937|
 168520844| 932|
 168659105| 930|
 168702106| 927|
 168551015| 924|
 168180650| 924|
 168721874| 922|
 167996379| 920|
 167798394| 918|
 1684023

In [26]:
all_play.groupby('uid').agg(func.countDistinct('song_id').alias('count')).orderBy('count', ascending = False).show(1000)

+---------+-----+
 uid|count|
+---------+-----+
168915463| 3610|
 45888463| 3189|
 69546515| 2983|
169031835| 2816|
167856245| 2608|
168179842| 2552|
168884001| 2398|
168377676| 2372|
168412815| 2362|
 1683241| 2143|
168278040| 2064|
 45365429| 1987|
168343454| 1973|
168224877| 1929|
168308159| 1863|
168611409| 1832|
168849784| 1831|
168275605| 1824|
168788690| 1815|
168287265| 1806|
168409118| 1789|
153049341| 1765|
167569705| 1750|
168163522| 1730|
167754128| 1644|
167791348| 1644|
168911999| 1640|
 42236906| 1638|
167718052| 1635|
167849958| 1629|
168251426| 1617|
167898102| 1616|
168418837| 1582|
167722439| 1582|
 43658018| 1575|
167884496| 1525|
168224985| 1498|
168451768| 1487|
168301741| 1485|
168702360| 1482|
168262301| 1455|
168309076| 1454|
168461660| 1437|
168714904| 1420|
167808656| 1417|
167839399| 1399|
168139162| 1397|
167805456| 1396|
168280987| 1396|
168988429| 1393|
168595606| 1353|
168836057| 1350|
168536012| 1336|
167765223| 1326|
168051193| 1319|
168483009| 1315|
168156556| 1310|
168529587| 1307|
168178214| 1305|
 33513144| 1301|
168250614| 1297|
168060852| 1289|
168938401| 1287|
168645368| 1287|
 62692715| 1286|
168066160| 1285|
167587977| 1281|
168368751| 1280|
168621058| 1279|
168639534| 1270|
 45312687| 1269|
167801421| 1268|
168356685| 1255|
168442115| 1254|
167880374| 1252|
168583145| 1251|
168157845| 1251|
 65116455| 1240|
168052947| 1236|
168155945| 1233|
168414339| 1231|
168355190| 1229|
167596452| 1226|
 58066741| 1219|
167772970| 1217|
168636306| 1212|
167691217| 1210|
168479098| 1209|
167737081| 1204|
168486395| 1186|
168497211| 1182|
168015735| 1181|
168542433| 1180|
168001008| 1180|
169038211| 1167|
168955269| 1167|
168751924| 1166|
167826027| 1151|
155180961| 1149|
167728979| 1148|
168045955| 1148|
168466886| 1143|
168289797| 1138|
168355686| 1129|
168755173| 1129|
167820900| 1127|
167739237| 1124|
168283992| 1121|
168909986| 1119|
167792930| 1108|
167967073| 1108|
 28146229| 1104|
168491032| 1103|
168027704| 1095|
168052175| 1091|
168192825| 1084|
168164135| 1080|
167775152| 1070|
167663420| 1066|
168606686| 1065|
167983827| 1060|
167727609| 1060|
168930795| 1057|
168954949| 1056|
168683103| 1055|
167910817| 1054|
168523030| 1053|
167834042| 1048|
168865280| 1048|
167758319| 1038|
168822909| 1033|
 25242167| 1030|
 60786265| 1027|
167575392| 1026|
168707975| 1023|
 42233120| 1022|
168387210| 1022|
168674358| 1015|
169041722| 1014|
168254653| 1014|
168658656| 1012|
168637523| 1005|
168462238| 1004|
168345192| 999|
167846506| 995|
168157255| 994|
167906119| 990|
167763847| 989|
168188112| 985|
168661744| 985|
168355960| 982|
169028588| 981|
167706804| 981|
167835950| 980|
168357183| 979|
169024129| 976|
168051149| 975|
168539380| 973|
167895141| 968|
168266329| 968|
168798094| 962|
168150083| 962|
169006057| 960|
168733141| 960|
167640838| 954|
168345004| 954|
167952240| 953|
168652062| 952|
168893573| 952|
168792308| 950|
167689442| 949|
168827140| 948|
164115461| 947|
 57820269| 940|
168726962| 940|
167776982| 939|
168006485| 937|
168520844| 932|
168659105| 930|
168702106| 927|
168180650| 924|
168551015| 924|
168721874| 922|
167996379| 920|
167798394| 918|
168402382| 917|
169011878| 916|
168980287| 913|
167845279| 910|
168662009| 909|
 88750693| 905|
167915578| 904|
168858790| 903|
168645999| 901|
168678913| 898|
167321380| 898|
167481248| 898|
168650801| 897|
168669274| 897|
168125280| 897|
167729598| 897|
168075893| 897|
169027226| 896|
168255392| 894|
168694211| 893|
168367107| 893|
167809388| 890|
 52569128| 884|
167963010| 883|
167892107| 882|
168260693| 880|
167846014| 877|
168825630| 876|
168843327| 875|
168916615| 875|
168150173| 874|
167955895| 872|
168059101| 871|
168887105| 868|
168055442| 868|
167795787| 867|
168455733| 865|
168232894| 862|
168725543| 860|
168302212| 858|
168539355| 857|
168268235| 857|
167885790| 855|
167732585| 855|
168940574| 853|
144491699| 853|
168930967| 851|
168200067| 850|
168517953| 850|
168977075| 849|
168637705| 845|
168034646| 844|
 24657226| 842|
16

In [27]:
# coding=utf-8
# HEADUP!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the thing i want to change, don't with column but change while filter
# also play time only need to use 1 or 1000 cutoff no need to change but the song length still need..(count how many needs the change, maybe not too many..then maybe want to only change the one need(only partition part of the sons) or maybe can use join, try jiahui's whether faster both due to gz and due to the song_length strategy, is she only using the ones need change?)
# uid thing, first abnormal uid, very close, then day of uid and then count songs, hopefully each uid with lots of songs, then by song count uid, drop songs with less thant 10 uid
# then partiontoin by song id to have three cutoff for ratings, maybe this can be joined with the above above step, both by uid count songs(the step could be by uid songid date first, then drop those smaller dates, or songs with samller uid, actaully the above can be just one steop)
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, FloatType
from pyspark.sql.functions import lit
from pyspark.sql.functions import trim
from pyspark.sql.functions import round
from pyspark.sql.functions import when
import pyspark.sql.functions as func
spark = SparkSession \
    .builder \
    .appName("music player") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
spark.catalog.clearCache()

schema_all_play = StructType([ StructField("uid", StringType(), True), StructField("device", StringType(), True),StructField("song_id", StringType(), True), StructField("song_type", StringType(), True), StructField("song_name", StringType(), True), StructField("singer", StringType(), True), StructField("play_time", StringType(), True),StructField("song_length", StringType(), True), StructField("paid_flag", StringType(), True), StructField("date", StringType(), True)])

schema_play = StructType([ StructField("uid", StringType(), True), StructField("device", StringType(), True),StructField("song_id", StringType(), True), StructField("song_type", StringType(), True), StructField("song_name", StringType(), True), StructField("singer", StringType(), True), StructField("play_time", StringType(), True),StructField("song_length", StringType(), True), StructField("paid_flag", StringType(), True)])

all_play = spark.createDataFrame([], schema = schema_all_play)

  
for path, name, _ in dbutils.fs.ls('/FileStore/tables/play/'):
# for path, name, _ in dbutils.fs.ls('/FileStore/tables/gz/'): 
  df = spark.read.csv(path, sep = "\t", schema = schema_play)
  date = name.split("_")[0].strip()
  df = df.withColumn("date",lit(date))

  all_play = all_play.unionAll(df)
# all_play.cache()
# print(all_play.count())  
all_play = all_play.withColumn('uid', trim(all_play.uid).cast(IntegerType()))
all_play = all_play.withColumn('song_id', trim(all_play.song_id).cast(IntegerType()))
all_play = all_play.withColumn('play_time', round(trim(all_play.play_time).cast(FloatType())) )
all_play = all_play.withColumn('song_length', round(trim(all_play.song_length).cast(FloatType())) )

all_play = all_play.filter( (all_play['play_time'] > 0.8 * all_play['song_length']) & (all_play.uid != 1685126) \
                    &(all_play.uid != 751824) \
                    &(all_play.uid != 37025504) \
                    &(all_play.uid != 1791497) \
                    &(all_play.uid != 1062806) \
                    &(all_play.uid != 497685) \
                    &(all_play.uid != 736305) \
                    &(all_play.uid > 99999) \
                    &(all_play.uid != 1749320) \
                    &(all_play.uid != 1679121) \
                    &(all_play.uid != 46532274) \
                    &(all_play.uid != 28638487) \
                    &(all_play.uid != 533817) \
                    &(all_play.uid != 155948236) \
                    &(all_play.uid != 637650) \
                    &(all_play.uid != 32166203) \
                    &(all_play.uid != 64268006) \
                    &(all_play.uid != 26036032) \
                    &(all_play.uid != 32104145) \
                    &(all_play.uid != 398309) \
                    &(all_play.uid != 154539052)\
                    & (all_play.uid != 16517426)\
                    & (all_play.uid != 27954505)\
                    & (all_play.uid != 168095004)\
                    & (all_play.uid != 1710083)\
                    & (all_play.uid != 1963913)\
                    & (all_play.uid != 924065)\
                    & (all_play.uid != 167615175)\
                    & (all_play.uid != 167848561)\
                    & (all_play.uid != 168565798)\
                    &(all_play.uid != 168510160) & (all_play.song_id > 9999) & (all_play['song_length'] > 80) & (all_play['play_time'] < 500 )  & (all_play['song_length'] < 480)  )
   
                          
all_play.cache()
print(all_play.count())
all_play.createOrReplaceTempView("all_play")

In [28]:
# coding=utf-8
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, FloatType
from pyspark.sql.functions import lit
from pyspark.sql.functions import trim
from pyspark.sql.functions import round
from pyspark.sql.functions import when
import pyspark.sql.functions as func
spark = SparkSession \
    .builder \
    .appName("music player") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
spark.catalog.clearCache()

schema_all_play = StructType([ StructField("uid", StringType(), True), StructField("device", StringType(), True),StructField("song_id", StringType(), True), StructField("song_type", StringType(), True), StructField("song_name", StringType(), True), StructField("singer", StringType(), True), StructField("play_time", StringType(), True),StructField("song_length", StringType(), True), StructField("paid_flag", StringType(), True), StructField("date", StringType(), True)])

schema_play = StructType([ StructField("uid", StringType(), True), StructField("device", StringType(), True),StructField("song_id", StringType(), True), StructField("song_type", StringType(), True), StructField("song_name", StringType(), True), StructField("singer", StringType(), True), StructField("play_time", StringType(), True),StructField("song_length", StringType(), True), StructField("paid_flag", StringType(), True)])

all_play = spark.createDataFrame([], schema = schema_all_play)

for path, name, _ in dbutils.fs.ls('/FileStore/tables/'):
# for path, name, _ in dbutils.fs.ls('/FileStore/tables/play/plays'):  
# for path, name, _ in dbutils.fs.ls('/FileStore/tables/play/'):  
  if name == "snap/" or name == "play/" or name == "plays/":
    break
  df = spark.read.csv(path, sep = "\t", schema = schema_play)
  date = name.split("_")[0].strip()
  df = df.withColumn("date",lit(date))

  all_play = all_play.unionAll(df)
  
all_play = all_play.withColumn('uid', trim(all_play.uid).cast(IntegerType()))
all_play = all_play.withColumn('song_id', trim(all_play.song_id).cast(IntegerType()))
all_play = all_play.withColumn('play_time', round(trim(all_play.play_time).cast(FloatType())) )
all_play = all_play.withColumn('song_length', round(trim(all_play.song_length).cast(FloatType())) )
# all_play = all_play.withColumn('date', trim(all_play.date))
# all_play = all_play.withColumn('song_name', trim(all_play.song_name))
# all_play = all_play.withColumn('singer', trim(all_play.singer))
all_play = all_play.filter( (all_play['play_time'] > 0.8 * all_play['song_length']) & (all_play.uid != 1685126) \
                    &(all_play.uid != 751824) \
                    &(all_play.uid != 37025504) \
                    &(all_play.uid != 1791497) \
                    &(all_play.uid != 1062806) \
                    &(all_play.uid != 497685) \
                    &(all_play.uid != 736305) \
                    &(all_play.uid > 99999) \
                    &(all_play.uid != 1749320) \
                    &(all_play.uid != 1679121) \
                    &(all_play.uid != 46532274) \
                    &(all_play.uid != 28638487) \
                    &(all_play.uid != 533817) \
                    &(all_play.uid != 155948236) \
                    &(all_play.uid != 637650) \
                    &(all_play.uid != 32166203) \
                    &(all_play.uid != 64268006) \
                    &(all_play.uid != 26036032) \
                    &(all_play.uid != 32104145) \
                    &(all_play.uid != 398309) \
                    &(all_play.uid != 154539052)\
                    & (all_play.uid != 167615175)\
                    &(all_play.uid != 168510160) & (all_play.song_id > 9999) & (all_play['song_length'] > 80) & (all_play['play_time'] < 500 )  & (all_play['song_length'] < 480)  )
all_play.cache()
print(all_play.count())
all_play.createOrReplaceTempView("all_play")

In [29]:
cc = all_play.groupby('uid', 'date').agg(func.count('song_id').alias('cnt'), (round((func.sum('play_time')/86400),1)).alias('sum'))
# .orderBy('sum', ascending = False)
dd = cc.groupby('uid').agg(func.countDistinct ('date').alias('cntD'), func.sum('cnt').alias('cntS') )
# ccc = cc.filter(cc['sum'] > 1)
# ccc.cache()
# ccc
ee = cc.filter(cc['sum'] > 1).groupby('uid').agg(func.sum('cnt').alias('cnts'),func.countDistinct ('date').alias('cnt_d'), func.first('sum').alias('top_sum'))
# .orderBy('cnt', ascending = False).show(1000)
f = ee.join(dd, ee.uid == dd.uid).select(dd.uid.alias('uid'), ee.top_sum.alias('top_sum'), dd.cntS.alias('cntS1'), ee.cnts.alias('cnts'), dd.cntD.alias('cntD1'), ee.cnt_d.alias('cnt_d'))
f = f.withColumn('pct1', f.cnts/f.cntS1)
f = f.withColumn('pct2', f.cnt_d/f.cntD1)
# ff = f.select('uid', ee.top_sum, dd.cntS, ee.cnts, ee.cnts/dd.cntS, dd.cntD, ee.cnt_d, ee.cnt_d/dd.cntD)
# .orderBy(ee.top_sum, ascending = False).filter(dd.cntD > 24)
f.select('uid').distinct().show(1000)

+----------+
 uid|
+----------+
 125169024|
134256529 |
154409183 |
154420309 |
154427603 |
154429096 |
154431112 |
154434558 |
154439476 |
154440708 |
154442375 |
154454363 |
154454895 |
154458253 |
154460806 |
154467259 |
154472575 |
154479534 |
154492586 |
154496857 |
154560207 |
154560786 |
154590126 |
154593899 |
154691344 |
154701910 |
154708543 |
154716495 |
154718183 |
154727074 |
154754641 |
154790122 |
154793582 |
154793599 |
154795068 |
 165374841|
167577077 |
 167589113|
 167590404|
167594755 |
167601730 |
167602360 |
 167618975|
 167620501|
167623395 |
167639436 |
167719511 |
 167742547|
167779410 |
 167841851|
 167872266|
 167879033|
 167929923|
167951283 |
 167975717|
 168006511|
168007255 |
 168008115|
 168029952|
168032175 |
 168063308|
 168064318|
 168071169|
 168071606|
 168074511|
 168075712|
 168082941|
 168093876|
 168099252|
 168107637|
 168232900|
 168276987|
 168294752|
 168297473|
 168319511|
 1683241|
 168333052|
 168333603|
 168333906|
 168402459|
 168414625|
 168475357|
 168510060|
 168510160|
 168533496|
 168553975|
 168566131|
 168581474|
 168635515|
 168637932|
 168645412|
 168646909|
 168657634|
 168666590|
 168683975|
 168713208|
 168729225|
 168732661|
 168734769|
 168735244|
 168739136|
 168752903|
 168761596|
 168800780|
 168801892|
 168812339|
 168818927|
 168821143|
 168837586|
 168848615|
 168848907|
 168862806|
 168865523|
 168894281|
 168921677|
 168936557|
 168963315|
 168985119|
 168988854|
 169005188|
 169005958|
 169011401|
 59179355|
127105880 |
 149875111|
154020338 |
154414949 |
154425393 |
154427293 |
154429244 |
154431468 |
154434899 |
154435681 |
154437570 |
154438561 |
154441057 |
154460803 |
154496772 |
154500180 |
154518917 |
154521785 |
154531716 |
154580807 |
154612952 |
154615486 |
154646578 |
154663756 |
154694790 |
154718185 |
154730343 |
154732079 |
154747506 |
154774594 |
154775009 |
154780432 |
154818610 |
154831458 |
154876850 |
 157910263|
167574268 |
167578065 |
 167578361|
167586793 |
 167596728|
167604192 |
 167608244|
 167608590|
167620838 |
167622492 |
167625222 |
167638382 |
 167702608|
 167765412|
 167849009|
 167863622|
 167905940|
 167945494|
167968520 |
 168035226|
 168070712|
 168073905|
 168084046|
 168084194|
 168085572|
 168085941|
 168104715|
 168160962|
 168161054|
 168215225|
 168270917|
 168309037|
 168364492|
 168375513|
 168442493|
 168467588|
 168534261|
 168537049|
 168570908|
 168572991|
 168577097|
168597954 |
 168599590|
 168607503|
 168621160|
 168626777|
 168629518|
 168650890|
 168695708|
 168708080|
 168711396|
 168733560|
 168749307|
 168751500|
 168765786|
 168775798|
 168834525|
 168835679|
 168845327|
 168875658|
 168885440|
 168886063|
 168892291|
 168918479|
 168929192|
 168949311|
 168957253|
 168958582|
 168964574|
 168982477|
 168983769|
 169021457|
 169025564|
 35092583|
 52593547|
 122470450|
154419976 |
154425375 |
154426381 |
154428697 |
154431673 |
154433060 |
154434766 |
154436899 |
154439281 |
154444341 |
154461634 |
154473397 |
154478723 |
154490539 |
154491396 |
154500407 |
154501682 |
154515874 |
154522072 |
154529379 |
154530523 |
154537076 |
154537642 |
154548414 |
154552050 |
154556594 |
154583950 |
154588063 |
154594044 |
154641505 |
154647395 |
154661573 |
154686210 |
154688038 |
154722118 |
154739170 |
154741716 |
154775517 |
154826319 |
154828528 |
 166999962|
167117807 |
 167573351|
 167581105|
 167584248|
 167586060|
167591971 |
 167595084|
 167595983|
167610887 |
167616847 |
 167617629|
 167621617|
 167637691|
 167678610|
 167683639|
167685593 |
 167723952|
 167778542|
 167800877|
167866553 |
 167933405|
 167952745|
 167961254|
167992331 |
 168007094|
168032417 |
168050934 |
 168065798|
 168070514|
 168076251|
 168087852|
 168090723|
 168095004|
 168101946|
 168108962|
 168182349|
 168248091|
 168274568|
 168282482|
 168295850|
 168330228|
 168335345|
 168487809|
 168514390|
 168532954|
 168546783|
 168554523|
 168560245|
 168574307|
 168580039|
 168581537|
 168603450|
 168633534|
 168640488|
 168648581|
 

In [30]:
dbutils.fs.mkdirs("FileStore/tables/play")
s = StructType([ StructField("uid", StringType(), True)] )
d = spark.read.csv('FileStore/tables/test1.csv', schema = s)

In [31]:
# coding=utf-8
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, FloatType
from pyspark.sql.functions import lit
from pyspark.sql.functions import trim
from pyspark.sql.functions import round
from pyspark.sql.functions import when
import pyspark.sql.functions as func
spark = SparkSession \
    .builder \
    .appName("music player") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
spark.catalog.clearCache()

schema_all_play = StructType([ StructField("uid", StringType(), True), StructField("device", StringType(), True),StructField("song_id", StringType(), True), StructField("song_type", StringType(), True), StructField("song_name", StringType(), True), StructField("singer", StringType(), True), StructField("play_time", StringType(), True),StructField("song_length", StringType(), True), StructField("paid_flag", StringType(), True), StructField("date", StringType(), True)])

schema_play = StructType([ StructField("uid", StringType(), True), StructField("device", StringType(), True),StructField("song_id", StringType(), True), StructField("song_type", StringType(), True), StructField("song_name", StringType(), True), StructField("singer", StringType(), True), StructField("play_time", StringType(), True),StructField("song_length", StringType(), True), StructField("paid_flag", StringType(), True)])

all_play = spark.createDataFrame([], schema = schema_all_play)

  
for path, name, _ in dbutils.fs.ls('/FileStore/tables/play/'):
# for path, name, _ in dbutils.fs.ls('/FileStore/tables/gz/'): 
  df = spark.read.csv(path, sep = "\t", schema = schema_play)
  date = name.split("_")[0].strip()
  df = df.withColumn("date",lit(date))

  all_play = all_play.unionAll(df)
# all_play.cache()
# print(all_play.count())  
# all_play = all_play.withColumn('uid', trim(all_play.uid).cast(IntegerType()))
# all_play = all_play.withColumn('song_id', trim(all_play.song_id).cast(IntegerType()))
# all_play = all_play.withColumn('play_time', round(trim(all_play.play_time).cast(FloatType())) )
# all_play = all_play.withColumn('song_length', round(trim(all_play.song_length).cast(FloatType())) )

all_play = all_play.filter( (all_play['play_time'] > 0.8 * all_play['song_length']) & (all_play.uid != 1685126) \
                    &(all_play.uid != 751824) \
                    &(all_play.uid != 37025504) \
                    &(all_play.uid != 1791497) \
                    &(all_play.uid != 1062806) \
                    &(all_play.uid != 497685) \
                    &(all_play.uid != 736305) \
                    &(all_play.uid > 99999) \
                    &(all_play.uid != 1749320) \
                    &(all_play.uid != 1679121) \
                    &(all_play.uid != 46532274) \
                    &(all_play.uid != 28638487) \
                    &(all_play.uid != 533817) \
                    &(all_play.uid != 155948236) \
                    &(all_play.uid != 637650) \
                    &(all_play.uid != 32166203) \
                    &(all_play.uid != 64268006) \
                    &(all_play.uid != 26036032) \
                    &(all_play.uid != 32104145) \
                    &(all_play.uid != 398309) \
                    &(all_play.uid != 154539052)\
                           
                    & (all_play.uid != 16517426)\
                    & (all_play.uid != 27954505)\                   
                    & (all_play.uid != 168095004)\
                    & (all_play.uid != 1710083)\       
                    & (all_play.uid != 1963913)\
                    & (all_play.uid != 924065)\
                    & (all_play.uid != 167615175)\
                    & (all_play.uid != 167848561)\      
                    & (all_play.uid != 168565798)\
                    &(all_play.uid != 168510160) & (all_play.song_id > 9999) & (all_play['song_length'] > 80) & (all_play['play_time'] < 500 )  & (all_play['song_length'] < 480)  )
#                     &(all_play.uid != 398309) \
#                     &(all_play.uid != 154539052)\
#                     
all_play.cache()
print(all_play.count())
all_play.createOrReplaceTempView("all_play")


46475153

In [32]:
cc = all_play.groupby('uid', 'date').agg(func.count('song_id').alias('cnt'), (round((func.sum('play_time')/86400),1)).alias('sum'))
dd = cc.groupby('uid').agg(func.countDistinct ('date').alias('cntD'), func.sum('cnt').alias('cntS') )
ee = cc.filter(cc['sum'] > 1).groupby('uid').agg(func.sum('cnt').alias('cnts'),func.countDistinct ('date').alias('cnt_d'), func.first('sum').alias('top_sum'))
f = ee.join(dd, ee.uid == dd.uid).select(dd.uid.alias('uid'), ee.top_sum.alias('top_sum'), dd.cntS.alias('cntS1'), ee.cnts.alias('cnts'), dd.cntD.alias('cntD1'), ee.cnt_d.alias('cnt_d'))
f = f.withColumn('pct1', f.cnts/f.cntS1)
f = f.withColumn('pct2', f.cnt_d/f.cntD1).filter(dd.cntD > 24).orderBy('cntS1', ascending = False)
# .orderBy('cntD1', ascending = False)

In [33]:
f.show(1000)
# 167613273 2373,  167981299 4688 .38 pct, 167848561 11311| 5205| 42| 7| 0.46 this denefintely need frop, 168791869| 1.1| 2507| 367| 35| 1| 0.14, |168965697| 2.1| 6927| 1622| 31| 1| 0.23, 
|168858307| 1.1| 2160| 356| 27| 1| 0.1648148148148148|0.037037037037037035| |168125418| 3.8| 3005| 1364| 33| 1| 0.45,
|168501892| 1.1| 2041| 366| 31| 1| 0.179
|168250921| 1.1| 9560| 2257| 38| 4| 0.2360878661087866| 0.10526315789473684| this one!
|168788495| 1.2| 2152| 403| 31| 1| 0.18
167977274| 1.2| 4032| 455| 38| 1| 0.112
|168238453| 1.5| 1544| 507| 25| 1| 0.32
169022311| 2.9| 1121| 936| 31| 1| 0.834

 27954505| 1.4|22858|20204| 42| 34| 0.8838
   1710083| 1.3|19829|13126| 42| 20| 0.66
    | 16517426| 1.5|23703|21957| 44| 38| 0.926

+---------+-------+-----+-----+-----+-----+--------------------+--------------------+
 uid|top_sum|cntS1| cnts|cntD1|cnt_d| pct1| pct2|
+---------+-------+-----+-----+-----+-----+--------------------+--------------------+
167849009| 1.3| 658| 492| 12| 1| 0.7477203647416414| 0.08333333333333333|
168364492| 1.3| 552| 472| 6| 1| 0.855072463768116| 0.16666666666666666|
167933405| 5.2| 2334| 1500| 16| 1| 0.6426735218508998| 0.0625|
168095004| 1.2| 644| 402| 19| 1| 0.6242236024844721| 0.05263157894736842|
167361553| 1.3| 1012| 474| 3| 1| 0.4683794466403162| 0.3333333333333333|
168602177| 1.2| 920| 350| 19| 1| 0.3804347826086957| 0.05263157894736842|
167704534| 1.1| 5513| 2116| 24| 3| 0.3838200616724107| 0.125|
168146094| 1.4| 738| 497| 5| 1| 0.6734417344173442| 0.2|
168175854| 1.2| 1020| 429| 15| 1| 0.42058823529411765| 0.06666666666666667|
168237334| 1.1| 1066| 379| 19| 1| 0.35553470919324576| 0.05263157894736842|
156340071| 2.4| 1192| 842| 8| 1| 0.7063758389261745| 0.125|
 93185093| 1.4| 2345| 518| 20| 1| 0.2208955223880597| 0.05|
168443592| 1.4| 524| 499| 6| 1| 0.9522900763358778| 0.16666666666666666|
167613273| 1.2| 2373| 404| 33| 1| 0.17024863042562158|0.030303030303030304|
168197715| 2.1| 1703| 1453| 8| 2| 0.8532002348796242| 0.25|
169016310| 1.1| 576| 402| 15| 1| 0.6979166666666666| 0.06666666666666667|
167687026| 7.1| 4158| 2495| 8| 1| 0.6000481000481| 0.125|
167990473| 1.3| 794| 451| 4| 1| 0.5680100755667506| 0.25|
168660048| 1.9| 2857| 1251| 18| 2| 0.4378718935946797| 0.1111111111111111|
167981299| 1.7| 4688| 1783| 26| 3| 0.38033276450511944| 0.11538461538461539|
167775589| 1.1| 585| 423| 3| 1| 0.7230769230769231| 0.3333333333333333|
167848561| 2.6|11311| 5205| 42| 7| 0.46017151445495535| 0.16666666666666666|
142837234| 1.5| 828| 505| 9| 1| 0.6099033816425121| 0.1111111111111111|
168791869| 1.1| 2507| 367| 35| 1| 0.14639010769844435| 0.02857142857142857|
168287004| 1.2| 811| 341| 12| 1| 0.4204685573366215| 0.08333333333333333|
168655647| 1.3| 1544| 1506| 7| 3| 0.9753886010362695| 0.42857142857142855|
 30546319| 2.1| 835| 777| 3| 1| 0.9305389221556887| 0.3333333333333333|
167905156| 2.5| 1021| 895| 5| 1| 0.8765915768854065| 0.2|
168373099| 1.1| 1160| 392| 17| 1| 0.33793103448275863|0.058823529411764705|
168248862| 1.5| 691| 523| 9| 1| 0.7568740955137482| 0.1111111111111111|
168824401| 1.8| 769| 607| 11| 1| 0.7893368010403121| 0.09090909090909091|
168959562| 2.1| 1539| 791| 9| 1| 0.5139701104613386| 0.1111111111111111|
168316429| 1.2| 697| 421| 9| 1| 0.6040172166427547| 0.1111111111111111|
140424531| 1.2| 413| 408| 2| 1| 0.9878934624697336| 0.5|
168965697| 2.1| 6927| 1622| 31| 1| 0.23415620037534285| 0.03225806451612903|
168112936| 1.6| 1238| 583| 17| 1| 0.47092084006462037|0.058823529411764705|
168858307| 1.1| 2160| 356| 27| 1| 0.1648148148148148|0.037037037037037035|
168125418| 3.8| 3005| 1364| 33| 1| 0.453910149750416|0.030303030303030304|
167930984| 3.4| 1515| 1124| 9| 1| 0.7419141914191419| 0.1111111111111111|
168052296| 1.5| 596| 475| 8| 1| 0.7969798657718121| 0.125|
168501892| 1.1| 2041| 366| 31| 1| 0.17932386085252328| 0.03225806451612903|
168397104| 1.1| 1123| 836| 10| 2| 0.7444345503116652| 0.2|
167751049| 2.9| 1048| 1048| 1| 1| 1.0| 1.0|
167887325| 1.1| 1770| 358| 17| 1| 0.20225988700564973|0.058823529411764705|
168300879| 1.2| 924| 413| 8| 1| 0.44696969696969696| 0.125|
168922198| 1.4| 1771| 486| 20| 1| 0.27442123094297005| 0.05|
167992299| 6.7| 1991| 1984| 3| 1| 0.9964841788046208| 0.3333333333333333|
168250921| 1.1| 9560| 2257| 38| 4| 0.2360878661087866| 0.10526315789473684|
168788495| 1.2| 2152| 403| 31| 1| 0.18726765799256506| 0.03225806451612903|
167874047| 1.5| 985| 984| 3| 2| 0.9989847715736041| 0.6666666666666666|
167977274| 1.2| 4032| 455| 38| 1| 0.11284722222222222| 0.02631578947368421|
168891450| 1.6| 3128| 566| 23| 1| 0.1809462915601023|0.043478260869565216|
167898128| 1.3| 479| 457| 3| 1| 0.954070981210856| 0.3333333333333333|
168628191| 1.3| 1801| 1237| 13| 2| 0.6868406440866185| 0.15384615384615385

In [34]:
f.show(1000)
| uid|top_sum|cntS1| cnts|cntD1|cnt_d| pct1| pct2| +---------+-------+-----+-----+-----+-----+--------------------+--------------------+ | 16517426| 1.5|23703|21957| 44| 38| 0.9263384381723833| 0.8636363636363636| | 27954505| 1.4|22858|20204| 42| 34| 0.8838918540554729| 0.8095238095238095| | 1710083| 1.3|19829|13126| 42| 20| 0.6619597559130567| 0.47619047619047616| | 1963913| 1.3|13206| 6605| 42| 12| 0.5001514463122823| 0.2857142857142857| |168565798| 1.2|11375| 413| 42| 1|0.036307692307692305|0.023809523809523808| |167848561| 2.6|11311| 5205| 42| 7| 0.46017151445495535| 0.16666666666666666| | 924065| 1.4| 9994| 4172| 37| 9| 0.4174504702821693| 0.24324324324324326|

+---------+-------+-----+-----+-----+-----+--------------------+--------------------+
 uid|top_sum|cntS1| cnts|cntD1|cnt_d| pct1| pct2|
+---------+-------+-----+-----+-----+-----+--------------------+--------------------+
 16517426| 1.5|23703|21957| 44| 38| 0.9263384381723833| 0.8636363636363636|
 27954505| 1.4|22858|20204| 42| 34| 0.8838918540554729| 0.8095238095238095|
 1710083| 1.3|19829|13126| 42| 20| 0.6619597559130567| 0.47619047619047616|
 1963913| 1.3|13206| 6605| 42| 12| 0.5001514463122823| 0.2857142857142857|
168565798| 1.2|11375| 413| 42| 1|0.036307692307692305|0.023809523809523808|
167848561| 2.6|11311| 5205| 42| 7| 0.46017151445495535| 0.16666666666666666|
 924065| 1.4| 9994| 4172| 37| 9| 0.4174504702821693| 0.24324324324324326|
167962526| 1.3| 9732| 1226| 33| 3| 0.12597616111796137| 0.09090909090909091|
168250921| 1.1| 9560| 2257| 38| 4| 0.2360878661087866| 0.10526315789473684|
 45365429| 1.1| 9156| 2232| 42| 5| 0.2437745740498034| 0.11904761904761904|
168884001| 1.1| 8953| 450| 42| 1|0.050262481849659334|0.023809523809523808|
168011385| 1.3| 7964| 1888| 30| 4| 0.2370668006027122| 0.13333333333333333|
167690625| 1.8| 7751| 533| 28| 1| 0.06876532060379306| 0.03571428571428571|
 69546515| 1.4| 7346| 1632| 42| 3| 0.22216172066430712| 0.07142857142857142|
168965697| 2.1| 6927| 1622| 31| 1| 0.23415620037534285| 0.03225806451612903|
168641844| 2.0| 6899| 1139| 38| 2| 0.16509639078127264| 0.05263157894736842|
168072772| 1.1| 6622| 740| 42| 2| 0.1117487163998792|0.047619047619047616|
168909986| 1.2| 6495| 417| 40| 1| 0.06420323325635104| 0.025|
168013316| 2.1| 5541| 790| 32| 1| 0.1425735426818264| 0.03125|
 17612166| 1.3| 5495| 370| 43| 1| 0.06733393994540492|0.023255813953488372|
167806662| 1.7| 5359| 648| 34| 1| 0.12091808173166635|0.029411764705882353|
168877387| 2.6| 4810| 865| 30| 1| 0.17983367983367984| 0.03333333333333333|
167981299| 1.7| 4688| 1783| 26| 3| 0.38033276450511944| 0.11538461538461539|
168283336| 1.6| 4608| 590| 32| 1| 0.12803819444444445| 0.03125|
168017780| 1.1| 4183| 412| 39| 1| 0.09849390389672484| 0.02564102564102564|
167977274| 1.2| 4032| 455| 38| 1| 0.11284722222222222| 0.02631578947368421|
167882094| 1.5| 3969| 1458| 40| 3| 0.3673469387755102| 0.075|
167682858| 1.9| 3754| 650| 33| 1| 0.17314864144912093|0.030303030303030304|
167696239| 3.9| 3683| 1337| 33| 1| 0.36301927776269344|0.030303030303030304|
167844226| 4.4| 3681| 1539| 28| 1| 0.4180929095354523| 0.03571428571428571|
 57820269| 1.1| 3584| 430| 40| 1| 0.11997767857142858| 0.025|
168284154| 1.4| 3563| 364| 28| 1| 0.10216110019646366| 0.03571428571428571|
168334631| 1.4| 3269| 1287| 30| 3| 0.39369837870908536| 0.1|
168337093| 2.7| 3184| 921| 29| 1| 0.28925879396984927|0.034482758620689655|
 22730453| 1.1| 3075| 617| 42| 1| 0.20065040650406504|0.023809523809523808|
168148350| 1.2| 3026| 387| 25| 1| 0.1278916060806345| 0.04|
168125418| 3.8| 3005| 1364| 33| 1| 0.453910149750416|0.030303030303030304|
168052175| 1.1| 2908| 1340| 27| 3| 0.4607977991746905| 0.1111111111111111|
167783330| 1.1| 2905| 888| 26| 2| 0.30567986230636834| 0.07692307692307693|
137084142| 1.1| 2741| 359| 33| 1| 0.13097409704487414|0.030303030303030304|
168276011| 1.5| 2644| 515| 27| 1| 0.1947806354009077|0.037037037037037035|
 24657226| 2.9| 2565| 1103| 37| 1| 0.4300194931773879| 0.02702702702702703|
168791869| 1.1| 2507| 367| 35| 1| 0.14639010769844435| 0.02857142857142857|
168151228| 1.6| 2460| 522| 27| 1| 0.2121951219512195|0.037037037037037035|
167613273| 1.2| 2373| 404| 33| 1| 0.17024863042562158|0.030303030303030304|
168283548| 1.3| 2284| 425| 37| 1| 0.18607705779334502| 0.02702702702702703|
167948289| 2.3| 2262| 780| 25| 1| 0.3448275862068966| 0.04|
168564593| 1.2| 2248| 424| 31| 1| 0.18861209964412812| 0.03225806451612903|
168858307| 1.1| 2160| 356| 27| 1| 0.1648148148148148|0.037037037037037035|
168470100| 5.1| 2155| 977| 28| 1| 0.45336426914153133| 0.03571428571428571|
168788495| 1.2| 2152| 403| 31| 1| 0.18726765799256506| 0.03225806451612903|
168371227| 1.2| 20

In [35]:
f.select('uid').distinct().show(1000)

+---------+
 uid|
+---------+
167849009|
168364492|
167933405|
168095004|
167361553|
168602177|
167704534|
168146094|
168175854|
168237334|
156340071|
 93185093|
168443592|
167613273|
168197715|
169016310|
167687026|
167990473|
168660048|
167981299|
167775589|
167848561|
142837234|
168791869|
168287004|
168655647|
 30546319|
167905156|
168373099|
168248862|
168824401|
168959562|
168316429|
140424531|
168965697|
168112936|
168858307|
168125418|
167930984|
168052296|
168501892|
168397104|
167751049|
167887325|
168300879|
168922198|
167992299|
168250921|
168788495|
167874047|
167977274|
168891450|
167898128|
168628191|
167579368|
167690625|
167714296|
168417403|
168238453|
 6073281|
 1404537|
167965150|
168781606|
167858241|
167888863|
 50771644|
168809836|
168521131|
168531686|
167689895|
168298282|
168931836|
168314817|
167783330|
 30045960|
168533982|
168018003|
168079224|
 24657226|
167111804|
167543234|
167806662|
168066648|
168212327|
169022311|
168067314|
168472236|
168679591|
168470100|
 17612166|
144919105|
168021273|
168276011|
168845964|
167799687|
168869626|
 27954505|
 28859085|
168018182|
168152749|
168395832|
168741262|
167899161|
168052175|
168612152|
168699627|
168564593|
 45242170|
167755621|
 22730453|
168335647|
169024395|
168814514|
168933401|
 924065|
168023223|
167706296|
 1710083|
167696239|
168635572|
168993784|
168946988|
168874105|
168013316|
167788694|
168126167|
167942557|
168771555|
168953271|
168151228|
168847484|
168813147|
 1963913|
168072772|
168284154|
168412626|
167577629|
167748346|
 57820269|
168331857|
168160117|
168374520|
168322120|
168420548|
168423671|
168661310|
168877387|
168830239|
168395453|
168588504|
 16517426|
167713919|
167980444|
168668668|
167912729|
168269508|
168481481|
167776180|
167800370|
167948289|
168193713|
168148350|
168884001|
167918368|
168603899|
168283548|
168341265|
167926549|
168167415|
168283336|
168303732|
168909986|
168008429|
169016556|
167568946|
168017780|
168011385|
168560934|
 6217429|
167655751|
168754768|
167693899|
167962526|
168698300|
168979416|
168583887|
167837115|
167882094|
168506944|
167683209|
106392460|
168633843|
137084142|
168565798|
168728131|
168966376|
168000459|
168830867|
168371227|
168486024|
 69546515|
167732245|
167957750|
167993131|
167844226|
168739009|
168892521|
169033185|
150738046|
168297162|
168920425|
168112966|
168227137|
167629129|
167817810|
167873990|
168038720|
168641844|
168762342|
168851672|
168491863|
168334631|
168470661|
168723617|
168337093|
 45365429|
165571131|
167708239|
167682858|
168957185|
167799821|
167910154|
168387354|
169014829|
167719405|
168015469|
168716206|
168342628|
+---------+

In [36]:
all_play.createOrReplaceTempView("all_play")

In [37]:
%sql
SELECT date, sum(if(trim(uid)=uid,0,1)) as sum1, sum(if(trim(song_id)=song_id,0,1)) as sum2, sum(if(trim(song_length)=song_id,0,1)) as sum3, sum(if(trim(play_time)=song_id,0,1)) as sum4
from all_play
group by date
order by sum1 desc, sum2 desc, sum3 desc, sum4 desc

date,sum1,sum2,sum3,sum4
20170301,3422930,3422936,3422936,3422936
20170302,2453601,2453607,2453607,2453607
20170339,2194479,2194482,2194482,2194482
20170303,1852995,1852999,1852999,1852999
20170304,1710065,1710065,1710065,1710065
20170305,1608947,1608947,1608947,1608947
20170306,1352351,1352351,1352351,1352351
20170307,1289449,1289449,1289449,1289449
20170309,1173766,1173766,1173766,1173766
20170331,2138,3041,7031208,6903167


In [38]:
x = all_play.groupby('song_id').agg(func.countDistinct('song_name').alias('d')).orderBy('d', ascending = False)

In [39]:
x.filter(x['d'] < 9).show(50)


+--------+---+
 song_id| d|
+--------+---+
 6916311| 8|
13761420| 8|
 3264466| 8|
 941528| 8|
 4855762| 8|
 8762277| 8|
 92564| 8|
 4147754| 8|
 6477241| 8|
13761415| 8|
 5844591| 8|
13761421| 8|
 441757| 7|
 160905| 7|
 7922570| 7|
 87955| 7|
 6657692| 7|
20870989| 7|
 6521982| 7|
 6202529| 7|
 1024952| 7|
 1244660| 7|
 506596| 7|
 8398741| 7|
 90100| 7|
 6536447| 7|
 8357238| 7|
 3372383| 7|
 6479608| 7|
 102820| 7|
 1028094| 7|
11689019| 7|
20869196| 7|
 5355765| 7|
 661304| 6|
 6191028| 6|
 6906525| 6|
 3398259| 6|
 9327383| 6|
15807836| 6|
 6310854| 6|
 458199| 6|
 6491920| 6|
 5335402| 6|
 6176032| 6|
 1106756| 6|
 5843759| 6|
 6311844| 6|
 1217815| 6|
 1534793| 6|
+--------+---+
only showing top 50 rows

In [40]:
all_play.filter(all_play['song_id'] == 13761415).select('song_name').distinct().show()

+--------------------+
 song_name|
+--------------------+
Audition(The Fool...|
Audition(The Fool...|
Audition(The Fool...|
Audition(The Fool...|
Audition(The Fool...|
Audition(The Fool...|
Audition(The Fool...|
Audition(The Fool...|
+--------------------+

In [41]:
%sql
SELECT DISTINCT SONG_NAME
FROM ALL_PLAY
WHERE SONG_ID = 13761415

SONG_NAME
Audition(The Fools Who Dream)-From ＂La La Land＂ Soundtrack
"Audition(The Fools Who Dream)-From "";La La Land""; Soundtrack"
"Audition(The Fools Who Dream)-From ""La La Land"" Soundtrack"
"Audition(The Fools Who Dream)-From \""La La Land\"" Soundtrack"
"Audition(The Fools Who Dream)-From ""La La Land"" Soundtrack"
"Audition(The Fools Who Dream)-From "";La La Land""; Soundtrack"
Audition(The Fools Who Dream)-From ＂La La Land＂ Soundtrack
Audition(The Fools Who Dream)


In [42]:
all_play.filter(all_play.song_name == "刚好遇见你" ).agg(func.countDistinct('uid')).show()

+-------------------+
count(DISTINCT uid)|
+-------------------+
 79629|
+-------------------+

In [43]:
all_play.agg(func.countDistinct('uid')).show()

+-------------------+
count(DISTINCT uid)|
+-------------------+
 616203|
+-------------------+

In [44]:
all_play.filter(all_play.play_time > 480).groupby('song_length').count().orderBy('song_length', ascending = False).show(400)

+-----------+-----+
song_length|count|
+-----------+-----+
 479.0| 6|
 477.0| 2|
 475.0| 3|
 470.0| 1|
 468.0| 1|
 456.0| 1|
 451.0| 1|
 450.0| 1|
 449.0| 3|
 448.0| 1|
 447.0| 1|
 445.0| 1|
 440.0| 1|
 439.0| 1|
 435.0| 1|
 433.0| 1|
 429.0| 1|
 427.0| 1|
 424.0| 1|
 423.0| 1|
 421.0| 1|
 419.0| 2|
 412.0| 1|
 410.0| 1|
 409.0| 1|
 406.0| 1|
 405.0| 4|
 403.0| 1|
 402.0| 3|
 401.0| 1|
 399.0| 3|
 398.0| 2|
 396.0| 2|
 395.0| 1|
 394.0| 1|
 393.0| 1|
 391.0| 2|
 390.0| 2|
 388.0| 3|
 387.0| 1|
 384.0| 1|
 383.0| 1|
 382.0| 1|
 380.0| 1|
 378.0| 1|
 377.0| 1|
 367.0| 3|
 366.0| 1|
 365.0| 1|
 363.0| 1|
 362.0| 1|
 361.0| 3|
 359.0| 2|
 357.0| 3|
 354.0| 3|
 350.0| 1|
 349.0| 1|
 348.0| 1|
 347.0| 2|
 346.0| 1|
 345.0| 1|
 343.0| 3|
 341.0| 2|
 338.0| 1|
 337.0| 1|
 334.0| 1|
 333.0| 8|
 332.0| 1|
 331.0| 1|
 330.0| 2|
 329.0| 1|
 328.0| 4|
 327.0| 3|
 325.0| 4|
 324.0| 3|
 322.0| 3|
 321.0| 3|
 320.0| 6|
 318.0| 6|
 315.0| 1|
 314.0| 2|
 313.0| 3|
 312.0| 8|
 311.0| 4|
 310.0| 4|
 309.0| 1|
 308.0| 2|
 307.0| 3|
 306.0| 8|
 305.0| 2|
 304.0| 1|
 303.0| 3|
 302.0| 4|
 301.0| 9|
 300.0| 8|
 299.0| 1|
 298.0| 9|
 297.0| 2|
 296.0| 5|
 295.0| 6|
 294.0| 3|
 293.0| 2|
 292.0| 4|
 291.0| 3|
 290.0| 5|
 289.0| 6|
 288.0| 5|
 287.0| 9|
 286.0| 5|
 285.0| 3|
 284.0| 2|
 283.0| 5|
 282.0| 7|
 281.0| 5|
 280.0| 7|
 279.0| 4|
 278.0| 3|
 277.0| 6|
 276.0| 4|
 275.0| 7|
 274.0| 4|
 273.0| 4|
 272.0| 4|
 271.0| 1|
 270.0| 7|
 269.0| 6|
 268.0| 13|
 267.0| 10|
 266.0| 9|
 265.0| 5|
 264.0| 7|
 263.0| 5|
 262.0| 9|
 261.0| 8|
 260.0| 8|
 259.0| 6|
 258.0| 8|
 257.0| 8|
 256.0| 9|
 255.0| 7|
 254.0| 5|
 253.0| 5|
 252.0| 2|
 251.0| 8|
 250.0| 5|
 249.0| 10|
 248.0| 9|
 247.0| 4|
 246.0| 2|
 245.0| 9|
 244.0| 6|
 243.0| 6|
 242.0| 6|
 241.0| 4|
 240.0| 1|
 239.0| 2|
 238.0| 2|
 237.0| 8|
 236.0| 8|
 235.0| 12|
 234.0| 3|
 233.0| 9|
 232.0| 4|
 231.0| 12|
 230.0| 8|
 229.0| 7|
 228.0| 8|
 227.0| 5|
 226.0| 8|
 225.0| 6|
 224.0| 5|
 223.0| 4|
 222.0| 4|
 221.0| 3|
 220.0| 4|
 219.0| 3|
 218.0| 6|
 217.0| 5|
 216.0| 5|
 215.0| 7|
 214.0| 4|
 213.0| 5|
 212.0| 9|
 211.0| 3|
 210.0| 4|
 209.0| 2|
 208.0| 1|
 207.0| 4|
 206.0| 4|
 205.0| 7|
 204.0| 4|
 203.0| 4|
 202.0| 2|
 201.0| 2|
 200.0| 8|
 199.0| 3|
 198.0| 2|
 197.0| 5|
 196.0| 1|
 195.0| 2|
 194.0| 3|
 193.0| 5|
 192.0| 4|
 191.0| 1|
 190.0| 2|
 187.0| 2|
 186.0| 2|
 185.0| 1|
 184.0| 3|
 183.0| 1|
 180.0| 2|
 177.0| 1|
 176.0| 1|
 174.0| 1|
 172.0| 1|
 171.0| 2|
 168.0| 1|
 166.0| 2|
 163.0| 1|
 161.0| 2|
 159.0| 1|
 157.0| 2|
 156.0| 2|
 154.0| 2|
 152.0| 1|
 151.0| 2|
 149.0| 2|
 148.0| 1|
 147.0| 1|
 144.0| 1|
 141.0| 1|
 139.0| 1|
 135.0| 1|
 132.0| 3|
 130.0| 2|
 128.0| 1|
 125.0| 5|
 124.0| 1|
 121.0| 1|
 120.0| 1|
 116.0| 1|
 115.0| 1|
 113.0| 1|
 111.0| 1|
 109.0| 1|
 102.0| 1|
 99.0| 1|
 92.0| 2|
 91.0| 1|
 89.0| 1|
 85.0| 2|
 82.0| 2|
+-----------+-----+

In [45]:
all_play.filter(all_play.play_time > 480).groupby('date').count().orderBy('date', ascending = False).show(60)

+--------+-----+
 date|count|
+--------+-----+
20170512| 9|
20170511| 7|
20170510| 10|
20170509| 10|
20170508| 21|
20170507| 8|
20170506| 8|
20170505| 11|
20170504| 18|
20170503| 6|
20170502| 18|
20170501| 11|
20170430| 13|
20170429| 23|
20170428| 12|
20170427| 13|
20170426| 7|
20170425| 9|
20170424| 6|
20170423| 14|
20170422| 20|
20170421| 7|
20170420| 6|
20170419| 8|
20170418| 11|
20170417| 9|
20170416| 12|
20170415| 16|
20170414| 14|
20170413| 15|
20170412| 20|
20170411| 14|
20170410| 11|
20170409| 22|
20170408| 20|
20170407| 23|
20170406| 14|
20170405| 14|
20170404| 17|
20170403| 9|
20170402| 26|
20170401| 17|
20170339| 8|
20170331| 43|
20170330| 16|
20170309| 7|
20170307| 21|
20170306| 6|
20170305| 5|
20170304| 12|
20170303| 16|
20170302| 19|
20170301| 146|
+--------+-----+

In [46]:
all_play.filter(all_play.play_time > 480).groupby('song_length').count().orderBy('count', ascending = False).show(400)

+-----------+-----+
song_length|count|
+-----------+-----+
 268.0| 13|
 235.0| 12|
 231.0| 12|
 249.0| 10|
 267.0| 10|
 256.0| 9|
 301.0| 9|
 266.0| 9|
 262.0| 9|
 298.0| 9|
 245.0| 9|
 233.0| 9|
 287.0| 9|
 248.0| 9|
 212.0| 9|
 261.0| 8|
 200.0| 8|
 228.0| 8|
 258.0| 8|
 312.0| 8|
 236.0| 8|
 257.0| 8|
 333.0| 8|
 306.0| 8|
 226.0| 8|
 300.0| 8|
 230.0| 8|
 260.0| 8|
 237.0| 8|
 251.0| 8|
 205.0| 7|
 215.0| 7|
 264.0| 7|
 275.0| 7|
 229.0| 7|
 270.0| 7|
 282.0| 7|
 255.0| 7|
 280.0| 7|
 269.0| 6|
 295.0| 6|
 225.0| 6|
 242.0| 6|
 318.0| 6|
 320.0| 6|
 259.0| 6|
 243.0| 6|
 244.0| 6|
 277.0| 6|
 479.0| 6|
 289.0| 6|
 218.0| 6|
 197.0| 5|
 125.0| 5|
 216.0| 5|
 193.0| 5|
 288.0| 5|
 224.0| 5|
 296.0| 5|
 283.0| 5|
 254.0| 5|
 265.0| 5|
 227.0| 5|
 281.0| 5|
 217.0| 5|
 290.0| 5|
 250.0| 5|
 253.0| 5|
 286.0| 5|
 263.0| 5|
 213.0| 5|
 292.0| 4|
 203.0| 4|
 192.0| 4|
 273.0| 4|
 276.0| 4|
 310.0| 4|
 207.0| 4|
 206.0| 4|
 311.0| 4|
 223.0| 4|
 210.0| 4|
 220.0| 4|
 204.0| 4|
 405.0| 4|
 272.0| 4|
 232.0| 4|
 274.0| 4|
 279.0| 4|
 214.0| 4|
 247.0| 4|
 222.0| 4|
 328.0| 4|
 241.0| 4|
 325.0| 4|
 302.0| 4|
 324.0| 3|
 313.0| 3|
 399.0| 3|
 294.0| 3|
 322.0| 3|
 132.0| 3|
 388.0| 3|
 354.0| 3|
 475.0| 3|
 307.0| 3|
 327.0| 3|
 184.0| 3|
 449.0| 3|
 361.0| 3|
 291.0| 3|
 234.0| 3|
 367.0| 3|
 194.0| 3|
 211.0| 3|
 199.0| 3|
 343.0| 3|
 357.0| 3|
 303.0| 3|
 221.0| 3|
 219.0| 3|
 321.0| 3|
 278.0| 3|
 402.0| 3|
 285.0| 3|
 202.0| 2|
 398.0| 2|
 305.0| 2|
 347.0| 2|
 391.0| 2|
 157.0| 2|
 314.0| 2|
 209.0| 2|
 154.0| 2|
 190.0| 2|
 195.0| 2|
 308.0| 2|
 151.0| 2|
 180.0| 2|
 161.0| 2|
 82.0| 2|
 187.0| 2|
 239.0| 2|
 166.0| 2|
 186.0| 2|
 198.0| 2|
 130.0| 2|
 341.0| 2|
 201.0| 2|
 419.0| 2|
 396.0| 2|
 330.0| 2|
 238.0| 2|
 390.0| 2|
 252.0| 2|
 149.0| 2|
 284.0| 2|
 359.0| 2|
 171.0| 2|
 293.0| 2|
 85.0| 2|
 92.0| 2|
 246.0| 2|
 156.0| 2|
 297.0| 2|
 477.0| 2|
 362.0| 1|
 331.0| 1|
 332.0| 1|
 196.0| 1|
 144.0| 1|
 89.0| 1|
 163.0| 1|
 113.0| 1|
 99.0| 1|
 102.0| 1|
 448.0| 1|
 349.0| 1|
 128.0| 1|
 148.0| 1|
 183.0| 1|
 124.0| 1|
 271.0| 1|
 429.0| 1|
 91.0| 1|
 174.0| 1|
 350.0| 1|
 384.0| 1|
 440.0| 1|
 315.0| 1|
 152.0| 1|
 365.0| 1|
 406.0| 1|
 366.0| 1|
 409.0| 1|
 299.0| 1|
 378.0| 1|
 185.0| 1|
 116.0| 1|
 363.0| 1|
 337.0| 1|
 394.0| 1|
 329.0| 1|
 445.0| 1|
 109.0| 1|
 424.0| 1|
 470.0| 1|
 191.0| 1|
 139.0| 1|
 240.0| 1|
 423.0| 1|
 309.0| 1|
 427.0| 1|
 387.0| 1|
 393.0| 1|
 141.0| 1|
 120.0| 1|
 176.0| 1|
 338.0| 1|
 410.0| 1|
 456.0| 1|
 168.0| 1|
 451.0| 1|
 348.0| 1|
 377.0| 1|
 468.0| 1|
 172.0| 1|
 435.0| 1|
 346.0| 1|
 450.0| 1|
 345.0| 1|
 395.0| 1|
 304.0| 1|
 147.0| 1|
 115.0| 1|
 380.0| 1|
 403.0| 1|
 439.0| 1|
 421.0| 1|
 401.0| 1|
 111.0| 1|
 135.0| 1|
 208.0| 1|
 433.0| 1|
 412.0| 1|
 177.0| 1|
 447.0| 1|
 159.0| 1|
 121.0| 1|
 382.0| 1|
 383.0| 1|
 334.0| 1|
+-----------+-----+

In [47]:
all_play.filter( (all_play['song_length'] < 479)&(all_play['song_length'] > 459)  & (all_play['play_time'] > 480) ).show()

+---------+------+--------+---------+--------------------+------+---------+-----------+---------+--------+
 uid|device| song_id|song_type| song_name|singer|play_time|song_length|paid_flag| date|
+---------+------+--------+---------+--------------------+------+---------+-----------+---------+--------+
154494925| ar | 3491999| 0 | 童话-恐龙的故事 | 儿童读物 | 499.0| 475.0| 0|20170301|
154494925| ar | 3491999| 0 | 童话-恐龙的故事 | 儿童读物 | 499.0| 475.0| 0|20170301|
154494925| ar | 3491999| 0 | 童话-恐龙的故事 | 儿童读物 | 499.0| 475.0| 0|20170301|
167760193| ar| 5892602| 0|Dj - 加快版节奏感超强中文舞曲...| 网络歌手| 489.0| 477.0| 0|20170405|
167760193| ar| 5892602| 0|Dj - 加快版节奏感超强中文舞曲...| 网络歌手| 489.0| 477.0| 0|20170406|
167960592| ar| 288588| 0| 夕阳箫鼓| 琵琶| 492.0| 468.0| 0|20170411|
168328790| ar|22978678| 0| 【小夜曲】我心里住了一个人| 米莉姑娘| 493.0| 470.0| 0|20170508|
+---------+------+--------+---------+--------------------+------+---------+-----------+---------+--------+

In [48]:
all_play.filter( (all_play['song_length'] == 268) & (all_play['play_time'] > 480) ).show()

+---------+------+--------+---------+--------------------+------+---------+-----------+---------+--------+
 uid|device| song_id|song_type| song_name|singer|play_time|song_length|paid_flag| date|
+---------+------+--------+---------+--------------------+------+---------+-----------+---------+--------+
154418068| ar | 6655009| 0 | 我们都不坏 | 乔任梁 | 481.0| 268.0| 0|20170301|
154418068| ar | 6655009| 0 | 我们都不坏 | 乔任梁 | 481.0| 268.0| 0|20170301|
167866231| ar| 6657692| 0| 走着走着就散了| 庄心妍| 483.0| 268.0| 0|20170331|
168574132| ar|23495106| 0|以人民的名义-(电视剧《人民的名义...|江映蓉&韩磊| 488.0| 268.0| 0|20170331|
168384836| ar| 6657692| 0| 走着走着就散了| 庄心妍| 485.0| 268.0| 0|20170401|
169013673| ar| 93157| 0| 江南| 林俊杰| 491.0| 268.0| 0|20170405|
169013673| ar| 93157| 0| 江南| 林俊杰| 491.0| 268.0| 0|20170406|
168372921| ar| 328058| 0| 思慕的人| 蔡琴| 497.0| 268.0| 0|20170408|
168302424| ar|16827764| 0| 八十年代的歌| 赵雷| 485.0| 268.0| 0|20170409|
168361966| ar| 6121432| 1| 错过了缘分错过你(发烧版)| 陈瑞| 485.0| 268.0| 0|20170412|
167572104| ar| 6657692| 0| 走着走着就散了| 庄心妍| 489.0| 268.0| 0|20170417|
168542621| ar| 513506| 0| 给自己的情书| 王菲| 492.0| 268.0| 0|20170422|
168489091| ar| 93157| 0| 江南| 林俊杰| 492.0| 268.0| 0|20170428|
+---------+------+--------+---------+--------------------+------+---------+-----------+---------+--------+

In [49]:
all_play.filter(all_play.uid == 154418068).groupby('date').count().show(500)

+--------+-----+
 date|count|
+--------+-----+
20170301| 105|
+--------+-----+

In [50]:
all_play.filter(all_play.uid == 154418068).groupby('play_time').count().show(500)

+---------+-----+
play_time|count|
+---------+-----+
 273.0| 1|
 261.0| 1|
 219.0| 1|
 192.0| 2|
 266.0| 2|
 158.0| 1|
 185.0| 1|
 264.0| 3|
 200.0| 1|
 162.0| 1|
 167.0| 1|
 265.0| 1|
 227.0| 2|
 221.0| 1|
 481.0| 84|
 215.0| 2|
+---------+-----+

In [51]:
all_play.filter( (all_play['play_time'] > 480) ).show(1000)

+---------+------+--------+---------+--------------------+--------------------+---------+-----------+---------+--------+
 uid|device| song_id|song_type| song_name| singer|play_time|song_length|paid_flag| date|
+---------+------+--------+---------+--------------------+--------------------+---------+-----------+---------+--------+
154418068| ar | 7222578| 0 | 少年游 | 魏晨 | 481.0| 194.0| 0|20170301|
154418068| ar | 6426603| 0 | 不将就-(电影《何以笙箫默》片尾曲) | 李荣浩 | 481.0| 312.0| 0|20170301|
154418068| ar | 945320| 0 | 不分手的恋爱 | 汪苏泷 | 481.0| 205.0| 0|20170301|
154418068| ar | 7594001| 0 |烈火神盾-(电视剧《麻辣变形计》主...| 魏晨 | 481.0| 193.0| 0|20170301|
154418068| ar | 6523910| 0 |假如爱有天意-(电影《太平轮·彼岸...| 李健 | 481.0| 303.0| 0|20170301|
154418068| ar | 243826| 0 | 龙卷风 | 周杰伦 | 481.0| 248.0| 0|20170301|
154418068| ar | 231534| 0 | 玫瑰花的葬礼 | 许嵩 | 481.0| 258.0| 0|20170301|
154418068| ar | 80403| 0 | 十年 | 陈奕迅 | 481.0| 206.0| 0|20170301|
154418068| ar | 1244660| 0 |有点甜-(电视剧《借用一下你的爱》...| 汪苏泷&By2 | 481.0| 235.0| 0|20170301|
154418068| ar | 7922570| 0 |微微一笑很倾城-(电视剧《微微一笑...| 杨洋 | 481.0| 204.0| 0|20170301|
154418068| ar | 4855762| 0 |青春修炼手册-(动漫《燃烧的蔬菜》...| TFBOYS | 481.0| 263.0| 0|20170301|
154418068| ar | 239082| 0 |爱情转移-(电影《爱情呼叫转移》主...| 陈奕迅 | 481.0| 260.0| 0|20170301|
154418068| ar | 6672344| 0 | 多幸运 | 韩安旭 | 481.0| 249.0| 0|20170301|
154418068| ar | 6561519| 0 |大梦想家-(电视剧《九州天空城》片...| TFBOYS | 481.0| 225.0| 0|20170301|
154418068| ar | 9817237| 0 | 爱是一个疯字 | 杨洋 | 481.0| 280.0| 0|20170301|
154418068| ar | 656637| 0 | 和你在一起 | 乔任梁 | 481.0| 287.0| 0|20170301|
154418068| ar | 6449273| 0 | 绅士 | 薛之谦 | 481.0| 290.0| 0|20170301|
154418068| ar | 228908| 0 | 晴天 | 周杰伦 | 481.0| 269.0| 0|20170301|
154418068| ar | 3611233| 0 |我们好像在哪见过{电视剧《咱们结婚...| 杨宗纬&叶蓓 | 481.0| 197.0| 0|20170301|
154418068| ar | 8762277| 0 |一笑倾城-(电视剧《微微一笑很倾城...| 汪苏泷 | 481.0| 231.0| 0|20170301|
154418068| ar | 4147754| 0 | 丑八怪-(电视剧《如果我爱你》插曲) | 薛之谦 | 481.0| 248.0| 0|20170301|
154418068| ar | 6660024| 0 | 勋章-(电影《我是证人》主题曲) | 鹿晗 | 481.0| 213.0| 0|20170301|
154418068| ar | 625784| 0 | 钻石 | 乔任梁 | 481.0| 255.0| 0|20170301|
154418068| ar | 6655009| 0 | 我们都不坏 | 乔任梁 | 481.0| 268.0| 0|20170301|
154418068| ar | 9932816| 0 | 一个人过 | 于朦胧 | 481.0| 294.0| 0|20170301|
154418068| ar | 9866208| 0 | 亲爱的你怎么不在我身边 | 张杰 | 481.0| 151.0| 0|20170301|
154418068| ar | 6248654| 0 | 贝加尔湖畔 | 李健 | 481.0| 318.0| 0|20170301|
154418068| ar | 9880667| 0 | 最美不过初相见 | 孙子涵 | 481.0| 282.0| 0|20170301|
154418068| ar | 9950164| 0 | 刚好遇见你 | 李玉刚 | 481.0| 200.0| 0|20170301|
154418068| ar | 6660691| 0 |一次就好-(电影《夏洛特烦恼》暖水曲) | 杨宗纬 | 481.0| 266.0| 0|20170301|
154418068| ar | 1105362| 0 | 单身情歌 | 林志炫 | 481.0| 277.0| 0|20170301|
154418068| ar | 7186129| 0 | 这就是爱 | 张杰&周一豪 | 481.0| 237.0| 0|20170301|
154418068| ar | 9880664| 0 | 傻傻等 | 孙子涵 | 481.0| 236.0| 0|20170301|
154418068| ar | 3514084| 0 | 模特 | 李荣浩 | 481.0| 306.0| 0|20170301|
154418068| ar | 6308715| 0 | 当你老了 | 李健 | 481.0| 320.0| 0|20170301|
154418068| ar | 6703175| 0 |一个人-(电影《前任2备胎反击战》...| 张艺兴 | 481.0| 231.0| 0|20170301|
154418068| ar | 2870532| 0 | 复活 | 乔任梁 | 481.0| 251.0| 0|20170301|
154418068| ar | 7149583| 0 | 告白气球 | 周杰伦 | 481.0| 215.0| 0|20170301|
154418068| ar | 7222578| 0 | 少年游 | 魏晨 | 481.0| 194.0| 0|20170301|
154418068| ar | 6426603| 0 | 不将就-(电影《何以笙箫默》片尾曲) | 李荣浩 | 481.0| 312.0| 0|20170301|
154418068| ar | 945320| 0 | 不分手的恋爱 | 汪苏泷 | 481.0| 205.0| 0|20170301|
154418068| ar | 7594001| 0 |烈火神盾-(电视剧《麻辣变形计》主...| 魏晨 | 481.0| 193.0| 0|20170301|
154418068| ar | 6523910| 0 |假如爱有天意-(电影《太平轮·彼岸...| 李健 | 481.0| 303.0| 0|20170301|
154418068| ar | 243826| 0 | 龙卷风 | 周杰伦 | 481.0| 248.0| 0|20170301|
154418068| ar | 231534| 0 | 玫瑰花的葬礼 | 许嵩 | 481.0| 258.0| 0|20170301|
154418068| ar | 80403| 0 | 十年 | 陈奕迅 | 481.0| 206.0| 0|20170301|
154418068| ar | 1244660| 0 |有点甜-(电视剧《借用一下你的爱》...| 汪苏泷&By2 | 481.0| 235.0| 0|20170301|
154418068| ar | 7922570| 0 |微微一笑很倾城-(电视剧《微微一笑...| 杨洋 | 481.0| 204.0| 0|20170301|
154418068| ar | 4855762| 0 |青春修炼手册-(动漫《燃烧的蔬菜》...| TFBOYS | 481.0| 263.0| 0|20170301|
154418068| ar | 239082| 0 |爱情转移-(电影《爱情呼叫转移》主...| 陈奕迅 | 481.0|

In [52]:
all_play.groupby(round(all_play['play_time']/all_play['song_length'], 1)).count().orderBy('count', ascending = False).show(500)

+-----------------------------------+--------+
round((play_time / song_length), 1)| count|
+-----------------------------------+--------+
 5.3| 1|
 6.1| 1|
 5.9| 1|
 5.1| 1|
 5.6| 1|
 5.8| 2|
 5.5| 2|
 5.0| 4|
 5.4| 4|
 4.9| 5|
 4.8| 5|
 4.5| 10|
 4.7| 10|
 4.2| 11|
 4.6| 15|
 4.4| 16|
 4.3| 18|
 4.1| 25|
 3.9| 26|
 3.8| 45|
 4.0| 52|
 3.6| 78|
 3.7| 85|
 3.5| 85|
 3.4| 145|
 3.3| 163|
 3.2| 206|
 3.1| 224|
 2.9| 304|
 3.0| 348|
 2.8| 476|
 2.7| 598|
 2.6| 788|
 2.5| 988|
 2.4| 1103|
 2.3| 1684|
 2.2| 2190|
 2.1| 2794|
 2.0| 3562|
 1.9| 5279|
 1.8| 5477|
 1.7| 6604|
 1.6| 8426|
 1.5| 11152|
 1.4| 14395|
 1.3| 21344|
 1.2| 32109|
 1.1| 53214|
 0.8| 1066796|
 0.9| 2259714|
 1.0|48394943|
+-----------------------------------+--------+

In [53]:
la = all_play.groupby('song_name').agg(func.countDistinct('uid').alias('cnt')).orderBy('cnt', ascending = False)

In [54]:
la.show(1000)

+--------------------+-----+
 song_name| cnt|
+--------------------+-----+
 刚好遇见你|79629|
凉凉-(电视剧《三生三世十里桃花》...|72670|
 逆流成河|58411|
 演员|52074|
 没有你陪伴真的好孤单|46445|
 成都|41205|
三生三世-(电视剧《三生三世十里桃...|40944|
 走着走着就散了|34897|
 告白气球|33503|
 你还要我怎样|33178|
 一人饮酒醉|28529|
 高尚|26945|
 Faded|25567|
 以后的以后|24948|
 勉为其难|24721|
思慕-(电视剧《三生三世十里桃花》...|24439|
 漂洋过海来看你|23431|
 丑八怪-(电视剧《如果我爱你》插曲)|23241|
 动物世界|23177|
 绅士|22621|
凉凉-(电视剧《三生三世十里桃花》...|22009|
 年轮-(电视剧《花千骨》插曲)|21505|
 刚好遇见你 |21035|
 刚刚好|20389|
 暧昧|19692|
 小时候|19691|
 千古-(电视剧《花千骨》片头曲)|19642|
 我只在乎你|18626|
 男孩|18496|
 See You Again|18333|
 一次就好-(电影《夏洛特烦恼》暖水曲)|18137|
 思慕(伴奏版)|17564|
 李白|17472|
 深夜书店|17298|
 当爱已成往事|16546|
 默-(电影《何以笙箫默》插曲)|16449|
 光辉岁月|16403|
 恋人心|16331|
 再见只是陌生人|16326|
 海阔天空|16029|
 西海情歌|16006|
 殇雪|15850|
 爱|15845|
 夜空中最亮的星|15841|
 我要醉在草原上|15784|
 凉凉|15672|
 Sugar|15479|
 皮皮虾我们走|15288|
 画心(Live)|15147|
 就算没有如果(伴奏版)|14877|
 江湖天下-(电视剧《射雕英雄传》插曲)|14857|
 甜蜜蜜|14790|
 裂心|14724|
以人民的名义-(电视剧《人民的名义...|14516|
 平凡之路|14111|
 逆战|14040|
 逆流成河 |14002|
 林中鸟|13957|
 歌在飞|13367|
三生三世-(电视剧《三生三世十里桃...|13292|
 吻别|13271|
 喜欢你 (原唱|13241|
 虫儿飞|13233|
 怒放的生命|13216|
 红玫瑰|13113|
 自己|13105|
 演员 |12816|
 认真的雪|12810|
 生来倔强|12790|
 一路上有你|12705|
 丑八怪|12694|
有点甜-(电视剧《借用一下你的爱》插曲)|12639|
 不分手的恋爱|12399|
 Good Time|11981|
 童话镇|11931|
 红尘情歌|11868|
 月半小夜曲|11757|
 走天涯|11682|
一笑倾城-(电视剧《微微一笑很倾城...|11528|
 我想我是海|11434|
 泡沫|11409|
 日落大道|11350|
 多幸运|11328|
 Rolling In The Deep|11294|
 你的过去我不介意|11219|
 没有你陪伴真的好孤单 |10969|
 宠爱|10817|
 等你等了那么久|10772|
 我的天空|10728|
 我和草原有个约定|10722|
 小幸运|10703|
 三十岁的女人 |10638|
 你给我听好|10593|
 我的好兄弟|10571|
 容易受伤的女人|10485|
 成都 |10243|
 红豆|10170|
骄傲的少年-(真人秀《高能少年团》...|10033|
 讲不出再见| 9859|
 爱河| 9755|
 心如刀割| 9754|
 把酒倒满| 9753|
 老男孩-(电影《老男孩》主题曲)| 9550|
 南山南| 9493|
拿不走的记忆-(电影《记忆大师》铭...| 9440|
 三生三世十里桃花| 9313|
 青花瓷| 9270|
 最炫民族风| 9240|
 白狐| 9150|
Writing's on the ...| 9017|
 致爱 Your Song| 9001|
 咱们结婚吧| 8973|
 一万个舍不得| 8942|
 社会摇| 8778|
 爱囚| 8726|
 天亮了| 8710|
 当爱情经过的时候| 8682|
 藤蔓之间| 8454|
 江南| 8391|
 长大以后的世界| 8260|
 难忘的一天| 8209|
 别把疼你的人弄丢了| 8200|
 红豆曲＋一生所爱| 7972|
 哥哥 | 7971|
 告白气球 | 7963|
 自由飞翔| 7828|
 十年| 7809|
 一千个伤心的理由| 7803|
 老街 (Live)| 7764|
小苹果-(电影《老男孩之猛龙过江》...| 7754|
 柿子| 7708|
匆匆那年-(电影《匆匆那年》同名主题曲)| 7699|
 千千阕歌| 7691|
我喜欢上你时的内心活动-(电影《喜...| 7656|
 最后的请求| 7623|
It's Love(Electro...| 7615|
 走着走着就散了 | 7611|
 独角戏| 7547|
 Call Me Maybe| 7542|
 慢慢习惯-(电影《拆弹专家》主题曲)| 7486|
 中国月亮| 7482|
 失落的缘-(电视剧《继承人》主题曲)| 7476|
微微一笑很倾城-(电视剧《微微一笑...| 7457|
 曾经的你| 7403|
 Love Yourself| 7394|
 Shape Of You| 7366|
 父亲| 7363|
 朋友的酒| 7341|
 一千万个拥抱| 7330|
 两个人的回忆一个人过| 7329|
 Fade| 7315|
 一往情深的恋人| 7302|
 模特| 7292|
 伤心城市| 7283|
 Animals| 7277|
 爱的世界只有你| 7266|
 等你等到我心痛| 7254|
 小芳| 7246|
 晴天| 7214|
 很想很想说再见 | 7210|
 父亲-(电影《父亲》主题曲)| 7136|
 勇气| 7129|
我的王国+We Will Rock...| 7127|
 别来纠缠我+Bleed It Out| 7078|
 爱是你我| 7055|
 雨花石| 7038|
 情书| 7033|
 断桥残雪| 6988|
 沉默是金| 6985|
 我的快乐就是想你| 6975|
 菊花爆满山| 6972|
 You Raise Me Up| 6966|
 好姑娘| 6965|
 爱江山更爱美人| 6964|
 大海| 6922|
 真的爱你| 6894|
 星辰-(电视剧《择天记》片头曲)| 6862|
 追梦人| 6853|
 丫头| 6743|
思慕-(电视剧《三生三世十里桃花》...| 6726|
 蓝莲花| 6669|
 你还要我怎样 | 6624|
 约定| 6591|
 大话西游| 6581|
 你就不要想起我| 6555|
下一秒-(电视剧《微微一笑很倾城》插曲)| 6554|
 If You| 6529|
 南方姑娘| 6517|
 独家记忆| 6469|
 有何不可| 6467|
 Bye Bye Bye| 6455|
 给我一个理由忘记| 6436|
 我害怕| 6431|
 玫瑰花的葬礼| 6382|
 漫步人生路| 6336|
 Bang Bang Bang| 6325|
 荷塘月色| 6315|
 水手| 6311|
 爱我你就抱抱我| 6255|
 离骚-(电视剧《思美人》片尾曲)| 6238|
 情人| 6232|
 那些花儿| 6225|
 青苹果乐园| 6213|
 我最亲爱的| 6197|
 哦啦啦啦| 6179|
 稻香| 6168|
 时间煮雨-(电影《小时代》主题曲)| 6168|
繁花-(电视剧《三生三世十里桃花》插曲)| 6165|
我是你的风-(动画电影《冰雪大作战...| 6122|
 守候-(电影《大闹天竺》主题曲)| 6091|
I Wanna Dance wit...| 6078|
 如果这都不算爱| 6077|
 小宝贝快快睡| 6032|
 Moves Like Jagger| 6022|
 假如| 6016|
 你在最近的天边| 5993|
 郎的诱惑| 5977|
 飘洋过海来看你| 5967|
 小幸运 (Live)| 5937|
 小酒窝| 5933|
 爱是怀疑| 5928|
大王叫我来巡山-(电影《万万没想到...| 5915|
 想你想疯了| 5867|
 一剪梅| 5860|
 鸿雁| 5851|
 Fantastic Baby| 5830|
 说散就散| 5799|
 天在下雨我在想你| 5791|
 新贵妃醉酒| 5781|
 擦肩而过| 5770|
 这条街| 5731|
 寻（《花儿与少年3·冒险季》主题曲）| 5701|
 女儿情| 5688|
 以后的以后 | 5688|
 Go Find A Man| 5686|
 珍珠-(电视剧《大唐荣耀》片尾曲)| 5673|
 Love Is Over| 5654|
 想着你亲爱的| 5653|
 注定-(电视剧《择天记》片尾曲)| 5649|
 伤不起| 5638|


In [55]:
la.createOrReplaceTempView('la')

In [56]:
%sql
SELECT *
FROM LA

song_name,cnt
刚好遇见你,79629
凉凉-(电视剧《三生三世十里桃花》片尾曲),72670
逆流成河,58411
演员,52074
没有你陪伴真的好孤单,46445
成都,41205
三生三世-(电视剧《三生三世十里桃花》主题曲),40944
走着走着就散了,34897
告白气球,33503
你还要我怎样,33178
